In [13]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [1]:
!pip3 install -q -U transformers
!pip3 install -q -U kaggle
!pip3 install -q -U accelerate
!pip3 install -q -U datasets


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.1 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.2 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which

In [5]:
from kaggle_secrets import UserSecretsClient
secret_label = "HF"
secret_value = UserSecretsClient().get_secret(secret_label)
from huggingface_hub import login
login(token=secret_value)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [14]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM,GenerationConfig,LEDForConditionalGeneration

In [16]:
model_id = "/kaggle/input/finetuned-kaggle-100-ckpt/checkpoint-100"

from transformers import AutoTokenizer, LEDModel
import torch

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = LEDForConditionalGeneration.from_pretrained(model_id).to("cuda:0")


In [20]:
def formatting_prompts_func(example):
    words=len(example["Output"].split())
#     print("no of words : " ,words)
#     print("---------"
    return  f'summarize the following data in {words} words: {example["Input"]}'


In [8]:
from datasets import load_dataset,Dataset

dataset = load_dataset("zqz979/meta-review", split="test")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [19]:
device = "cuda:0"
encodeds = tokenizer(formatting_prompts_func(dataset[2]), return_tensors="pt")


model_inputs = encodeds.to(device)

generated_ids = model.generate(**model_inputs)
decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(decoded)

no of words :  111
---------
This paper proposes to use Cluster Learnability (ID) and Cluster Learnable (CL) to evaluate the quality of self-supervised learning representations. The reviewers found that the proposed metrics are not sufficient to justify the proposed methods.    The reviewers also found that there was a lack of clarity on the experimental results.  The authors did not address the reviewers' concerns and the reviewers did not take into account the technical aspects of the paper. The authors have not addressed the reviewers’ concerns and concerns. 


In [21]:
dataset

Dataset({
    features: ['Input', 'Output'],
    num_rows: 1649
})

In [22]:
original_model_id ="allenai/led-base-16384"
original_tokenizer = AutoTokenizer.from_pretrained(original_model_id)
original_model = LEDForConditionalGeneration.from_pretrained(original_model_id).to("cuda:1")

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/648M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [23]:
# input_ids = tokenizer(dataset[0]["Input"], return_tensors="pt").input_ids.to("cuda:1")
# original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=300),repetition_penalty=10.0)
# original_model_text_output = original_tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

In [31]:
raw_input = dataset[0:10]
human_baseline_summaries = dataset[0:10]['Output']

original_model_summaries = []

led_model_summaries = []

for i in range(10):
    words=len((dataset[i]["Output"]).split())
    prompt1 =  f'summarize the following data in {words} words: {dataset[i]["Input"]}'

    input_ids1 = tokenizer(prompt1, return_tensors="pt").to("cuda:0")
    input_ids2 = original_tokenizer(dataset[i]["Input"], return_tensors="pt").to("cuda:1")
    human_baseline_text_output = human_baseline_summaries[i]

    
    led_model_outputs = model.generate(**input_ids1)
    led_model_text_output = tokenizer.decode(led_model_outputs[0], skip_special_tokens=True)
    
    original_model_outputs = original_model.generate(**input_ids2)
    original_model_text_output = original_tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)


    original_model_summaries.append(original_model_text_output)

    led_model_summaries.append(led_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, led_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])
df

Input ids are automatically padded from 2686 to 3072 to be a multiple of `config.attention_window`: 1024
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Input ids are automatically padded from 2736 to 3072 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 2332 to 3072 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 1801 to 2048 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 2612 to 3072 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 1139 to 2048 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 2033 to 2048 to be a multiple of `config.attenti

,human_baseline_summaries,original_model_summaries,peft_model_summaries
0,The paper present results using syntactic info...,This paper is a very interesting paper. It is ...,This paper presents a new framework for addres...
1,The authors focus on large scale out-of-distri...,The paper presents a large-scale evaluation of...,The reviewers found that the paper is well wri...
2,The problem studied in this paper is interesti...,This paper tries to give a measurement method ...,This paper proposes to use Cluster Learnabilit...
3,The major criticism of this paper after the in...,This paper proposes a new gradient attack meth...,This paper studies the problem of gradient att...
4,This paper proposes a novel challenge setting:...,This work proposed to evaluate the robustness ...,This paper investigates the robustness of Alph...
5,The paper explores generation of a volumetric ...,The work tackles the problem of 3D scene gener...,This paper presents a large scale scene genera...
6,The paper introduces the new task of few-shot ...,agnostic segmentation networks with iterative ...,This paper proposes a few-shot semantic edge d...
7,This paper studies the computational benefits ...,The paper considers the tasks of adversarial r...,This paper studies adversarial robustness. The...
8,The proposed approach exploits the color distr...,===========================================...,This paper introduces a new unrestricted adver...
9,The paper proposes a variant of recurrent neur...,In this paper the authors propose a novel arch...,This paper proposes a novel LSTM structure cal...


In [29]:
!pip3 install -q -U rouge_score
!pip3 install -q -U evaluate
import evaluate
rouge = evaluate.load('rouge')


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2024-03-13 05:04:07.449955: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 05:04:07.450052: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to re

In [32]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

led_model_summaries = rouge.compute(
    predictions=led_model_summaries,
    references=human_baseline_summaries[0:len(led_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('LED MODEL:')
print(led_model_summaries)

ORIGINAL MODEL:
{'rouge1': 0.10016438761209763, 'rouge2': 0.021009077142665797, 'rougeL': 0.07852940895224593, 'rougeLsum': 0.07867830373750495}
LED MODEL:
{'rouge1': 0.3256944249310342, 'rouge2': 0.08113142287139949, 'rougeL': 0.1914392524807772, 'rougeLsum': 0.19203892581924215}
